<a href="https://colab.research.google.com/github/gabrielefrn/qc-minicourse/blob/main/qc_minicourse_10abr24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum noise and error mitigation